LAB4: Deloy IBM assistance (chatbot) hỗ trợ tư vấn học đường lên web app (sử dụng colab + Flask + API IBM) + Kết nối database DB2 IBM thêm xoá danh sách sinh viên. Deadline 22/10 demo trực tiếp (submit code) @All

In [ ]:
!pip install ibm_db

In [ ]:
import ibm_db
import os
import pandas as pd

# Set your IBM Db2 credentials in the environment variables or manually assign them in the code below.
dsn_hostname = os.getenv('DB_HOST', 'null-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud') # change this
dsn_uid = os.getenv('DB_UID', 'null')  # change this
dsn_pwd = os.getenv('DB_PWD', 'null')  # change this
dsn_port = os.getenv('DB_PORT', 'null')  # change this
dsn_database = "bludb"  # change if necessary
dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_protocol = "TCPIP"
dsn_security = "SSL"

# $env:HOST_URL = "0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud"
# $env:USERNAME = "lyf46748"
# $env:PASSWORD = "FCMsRoJTA7TZ1SYx"
# $env:PORT = "31198"
# Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};"
    "SECURITY={7};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd, dsn_security)




In [ ]:
def get_data():
  try:
      conn = ibm_db.connect(dsn, "", "")
      print("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

      # Query to get the first 5 rows from the table
      command = "Select * from hello"

      stmt = ibm_db.exec_immediate(conn, command)

      # Fetch the result set
      result = ibm_db.fetch_both(stmt)
      rows = []

      # Loop through the result and fetch the rows
      while result:
          rows.append(result)
          result = ibm_db.fetch_both(stmt)

      # Convert result to a pandas DataFrame for better readability
      df = pd.DataFrame(rows)
      return df
  except Exception as e:
      print("Error connecting to the database: ", str(e))
  finally:
      # Close the connection
      if conn:
          ibm_db.close(conn)

In [ ]:
def filter_data():
  data = get_data()
  data1 = data.loc[:, ['Tên Sinh Viên', 'Ngày Sinh', 'Mã Số', 'Chuyên Ngành']]
  return data1

In [ ]:
import ibm_db

# Function to add a new student to the database
def add_student(id, name, major, birthday):
    conn = None
    try:
        conn = ibm_db.connect(dsn, "", "")
        print(f"Connected to database: {dsn_database} as user: {dsn_uid} on host: {dsn_hostname}")

        # Use parameterized queries to prevent SQL injection
        sql = 'INSERT INTO hello (\"Tên Sinh Viên\", \"Ngày Sinh\", \"Mã Số\", \"Chuyên Ngành\") VALUES (?, ?, ?, ?)'
        stmt = ibm_db.prepare(conn, sql)

        # Bind parameters
        ibm_db.bind_param(stmt, 1, name)
        ibm_db.bind_param(stmt, 2, birthday)
        ibm_db.bind_param(stmt, 3, id)
        ibm_db.bind_param(stmt, 4, major)

        # Execute the statement
        ibm_db.execute(stmt)
        print("Student added successfully!")

    except Exception as e:
        print("Error adding student to the database:", str(e))

    finally:
        if conn:
            try:
                ibm_db.close(conn)
            except Exception as e:
                print("Error closing the database connection:", str(e))


# Function to delete a student from the database by ID
def delete_student(id):
    try:
        conn = ibm_db.connect(dsn, "", "")
        print(f"Connected to database: {dsn_database} as user: {dsn_uid} on host: {dsn_hostname}")

        # Use parameterized queries to prevent SQL injection
        sql = "DELETE FROM hello WHERE \"Mã Số\" = ?"
        stmt = ibm_db.prepare(conn, sql)

        # Bind the parameter
        ibm_db.bind_param(stmt, 1, id)

        # Execute the statement
        ibm_db.execute(stmt)
        print("Student deleted successfully!")

    except Exception as e:
        print("Error deleting student from the database:", str(e))
    finally:
        if conn:
            ibm_db.close(conn)


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://0lwv7swztnh-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, request, render_template_string, redirect, url_for

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def hello():
    # Handle form submission for adding or deleting a student
    if request.method == 'POST':
        if 'add_student' in request.form:
            # Get data from the add student form
            id = request.form['id']
            name = request.form['name']
            major = request.form['major']
            birthday = request.form['birthday']
            # Add the student to the database
            add_student(id, name, major, birthday)
        elif 'delete_student' in request.form:
            # Get the ID from the delete student form
            id = request.form['delete_id']
            # Delete the student from the database
            delete_student(id)

        # Redirect to avoid form resubmission on refresh
        # return redirect(url_for('hello'))

    # Get the updated DataFrame
    df = filter_data()
    df_html = df.to_html(classes='dataframe', index=False)

    # HTML content with forms for adding and deleting students
    html_content = '''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Manage Students</title>
        <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;500&display=swap" rel="stylesheet">
        <style>
            body {
                font-family: 'Roboto', sans-serif;
                background-color: #f4f6f9;
                color: #333;
                margin: 0;
                padding: 0;
            }
            h1, h2 {
                text-align: center;
                margin-top: 20px;
                color: #333;
            }
            table.dataframe {
                width: 80%;
                margin: 20px auto;
                border-collapse: collapse;
                background-color: #fff;
                box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
            }
            table.dataframe, table.dataframe th, table.dataframe td {
                border: 1px solid #ddd;
            }
            table.dataframe th, table.dataframe td {
                padding: 10px;
                text-align: center;
            }
            table.dataframe th {
                background-color: #f2f2f2;
                font-weight: bold;
            }
            table.dataframe tr:hover {
                background-color: #f5f5f5;
            }
            .form-container {
                width: 50%;
                margin: 20px auto;
                padding: 20px;
                background-color: #fff;
                border-radius: 8px;
                box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
            }
            .form-container form {
                display: flex;
                flex-direction: column;
            }
            .form-container input[type="text"],
            .form-container input[type="date"] {
                padding: 10px;
                margin-bottom: 15px;
                border: 1px solid #ccc;
                border-radius: 4px;
                font-size: 16px;
            }
            .form-container button {
                padding: 10px;
                background-color: #007bff;
                color: white;
                border: none;
                border-radius: 4px;
                font-size: 16px;
                cursor: pointer;
                transition: background-color 0.3s;
            }
            .form-container button:hover {
                background-color: #0056b3;
            }
            .form-container label {
                font-weight: 500;
                margin-bottom: 5px;
                color: #555;
            }
        </style>
    </head>
    <body>
        <h1>Manage Students</h1>

        <!-- Display the DataFrame as an HTML table -->
        <div>
            {{ table | safe }}
        </div>

        <!-- Form to add a student -->
        <div class="form-container">
            <h2>Add Student</h2>
            <form method="POST">
                <label for="id">ID:</label>
                <input type="text" id="id" name="id" required>

                <label for="name">Name:</label>
                <input type="text" id="name" name="name" required>

                <label for="major">Major:</label>
                <input type="text" id="major" name="major" required>

                <label for="birthday">Birthday:</label>
                <input type="date" id="birthday" name="birthday" required>

                <button type="submit" name="add_student">Add Student</button>
            </form>
        </div>

        <!-- Form to delete a student -->
        <div class="form-container">
            <h2>Delete Student</h2>
            <form method="POST">
                <label for="delete_id">ID:</label>
                <input type="text" id="delete_id" name="delete_id" required>
                <button type="submit" name="delete_student">Delete Student</button>
            </form>
        </div>

        <!-- Watson Assistant Chat Script -->
        <script>
          window.watsonAssistantChatOptions = {
            integrationID: "null-5164-4400-8ae5-16224be6d559", // The ID of this integration.
            region: "au-syd", // The region your integration is hosted in.
            serviceInstanceID: "null-f6a6-4e50-a512-72a14329ef9e", // The ID of your service instance.
            onLoad: async (instance) => { await instance.render(); }
          };
          setTimeout(function(){
            const t=document.createElement('script');
            t.src="https://web-chat.global.assistant.watson.appdomain.cloud/versions/" +
            (window.watsonAssistantChatOptions.clientVersion || 'latest') + "/WatsonAssistantChatEntry.js";
            document.head.appendChild(t);
          });
        </script>
    </body>
    </html>
    '''
    return render_template_string(html_content, table=df_html)

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Connected to database:  bludb as user:  lyf46748 on host:  0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:24:13] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:24:15] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -


Connected to database: bludb as user: lyf46748 on host: 0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud
Student deleted successfully!
Connected to database:  bludb as user:  lyf46748 on host:  0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:25:20] "POST /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:25:21] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -


Connected to database: bludb as user: lyf46748 on host: 0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud
Error adding student to the database: Statement Execute Failed: [IBM][CLI Driver] CLI0109E  String data right truncation. SQLSTATE=22001 SQLCODE=-99999
Connected to database:  bludb as user:  lyf46748 on host:  0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:26:03] "POST /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:26:04] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -


Connected to database: bludb as user: lyf46748 on host: 0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud
Student added successfully!
Connected to database:  bludb as user:  lyf46748 on host:  0c77d6f2-5da9-48a9-81f8-86b520b87518.bs2io90l08kqb1od8lcg.databases.appdomain.cloud


INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:27:07] "POST /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [24/Oct/2024 01:27:08] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -
